# Financial analyst agent

The job of this agent is to build a financial report with investment analysis, business highlights, risk assessment and chart plotting and ultimately reporting all findings inside a PDF report for business leaders.

#### Tasks to be performed
- Create a financial analyst prompt
- Define tools to be used by the model

## TODO
- Ensure to set `FMP_API_KEY`, `FINNHUB_API_KEY`, `SEC_API_KEY`
- Having trouble with functions within the Utility classes. Perhaps I want to bring in the functions into the notebook and ultimately turn them to langchain tools(BaseTool and/or StructuredTool)

In [1]:
# %%capture --no-stderr
# !pip install -U langchain langchain-core langchain-community langgraph

In [1]:
import sys
import os

# Add the parent directory to sys.path
sys.path.insert(0, os.path.abspath('..'))

# Now you can import the module
from utils import register_keys_from_json

CONFIG_API_KEY_PATH = "../config_api_key"

# This runs important enviromental variables
register_keys_from_json(CONFIG_API_KEY_PATH)

In [2]:
# from pathlib import Path
# from tempfile import TemporaryDirectory

# _TEMP_DIRECTORY = TemporaryDirectory()
# WORKING_DIRECTORY = Path(_TEMP_DIRECTORY.name)
# WORKING_DIRECTORY

In [3]:
# Import classes holding the tools
from data_sources import FinnHubUtils, YFinanceUtils, FMPUtils, SECUtils
from functional import ReportAnalysisUtils, MplFinanceUtils, ReportChartUtils, ReportLabUtils, TextUtils

Report directory:  C:\Users\abdul\Desktop\git\alpha_agent\financial_annual_report
Report directory:  C:\Users\abdul\Desktop\git\alpha_agent\financial_annual_report


In [4]:
# Assign tools
get_sec_report = FMPUtils.get_sec_report
analyze_balance_sheet = ReportAnalysisUtils.analyze_balance_sheet
analyze_income_stmt, analyze_cash_flow = ReportAnalysisUtils.analyze_income_stmt, ReportAnalysisUtils.analyze_cash_flow
analyze_balance_sheet, analyze_segment_stmt = ReportAnalysisUtils.analyze_balance_sheet, ReportAnalysisUtils.analyze_segment_stmt
income_summarization = ReportAnalysisUtils.income_summarization
analyze_business_highlights = ReportAnalysisUtils.analyze_business_highlights
analyze_company_description = ReportAnalysisUtils.analyze_company_description
get_risk_assessment = ReportAnalysisUtils.get_risk_assessment
get_share_performance, get_pe_eps_performance = ReportChartUtils.get_share_performance, ReportChartUtils.get_pe_eps_performance
check_text_length = TextUtils.check_text_length
build_annual_report = ReportLabUtils.build_annual_report

In [5]:
from langchain_core.tools import StructuredTool

In [6]:
tool_functions = [
    get_sec_report,
    analyze_balance_sheet,
    analyze_income_stmt,
    analyze_cash_flow,
    analyze_balance_sheet,
    analyze_segment_stmt,
    income_summarization,
    analyze_business_highlights,
    analyze_company_description,
    get_risk_assessment,
    get_share_performance,
    get_pe_eps_performance,
    check_text_length,
    build_annual_report
]

tools = [StructuredTool.from_function(tool) for tool in tool_functions]

In [7]:
company = "Microsoft"
fyear = "2023"
# work_dir = WORKING_DIRECTORY

In [8]:
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

expert_investor_prompt = """
            Role: Expert Investor
            Department: Finance
            Primary Responsibility: Generation of Customized Financial Analysis Reports

            Role Description:
            As an Expert Investor within the finance domain, your expertise is harnessed to develop bespoke Financial Analysis Reports that cater to specific client requirements. This role demands a deep dive into financial statements and market data to unearth insights regarding a company's financial performance and stability. Engaging directly with clients to gather essential information and continuously refining the report with their feedback ensures the final product precisely meets their needs and expectations.

            Key Objectives:

            Analytical Precision: Employ meticulous analytical prowess to interpret financial data, identifying underlying trends and anomalies.
            Effective Communication: Simplify and effectively convey complex financial narratives, making them accessible and actionable to non-specialist audiences.
            Client Focus: Dynamically tailor reports in response to client feedback, ensuring the final analysis aligns with their strategic objectives.
            Adherence to Excellence: Maintain the highest standards of quality and integrity in report generation, following established benchmarks for analytical rigor.
            Performance Indicators:
            The efficacy of the Financial Analysis Report is measured by its utility in providing clear, actionable insights. This encompasses aiding corporate decision-making, pinpointing areas for operational enhancement, and offering a lucid evaluation of the company's financial health. Success is ultimately reflected in the report's contribution to informed investment decisions and strategic planning.

            Based on your description above, you are to do the following:

            With the tools you've been provided, write an annual report based on {company}'s {fyear} 10-k report, format it into a pdf.
            Pay attention to the followings:
            - Explicitly explain your working plan before you kick off.
            - Use tools one by one for clarity, especially when asking for instructions. 
            - Display any image in the chat once generated.
            - All the paragraphs should combine between 400 and 450 words, don't generate the pdf until this is explicitly fulfilled.
            
            Reply TERMINATE when everything is settled.
            """

# prompt = PromptTemplate(
#         template=expert_investor_prompt,
#         input_variables=["company", "fyear", "work_dir"],
#     )

expert_investor_system_prompt = """
            Role: Expert Investor
            Department: Finance
            Primary Responsibility: Generation of Customized Financial Analysis Reports

            Role Description:
            As an Expert Investor within the finance domain, your expertise is harnessed to develop bespoke Financial Analysis Reports that cater to specific client requirements. 
            This role demands a deep dive into financial statements and market data to unearth insights regarding a company's financial performance and stability. 
            Engaging directly with clients to gather essential information and continuously refining the report with their feedback ensures the final product precisely meets their needs and expectations.

            Key Objectives:

            Analytical Precision: Employ meticulous analytical prowess to interpret financial data, identifying underlying trends and anomalies.
            Effective Communication: Simplify and effectively convey complex financial narratives, making them accessible and actionable to non-specialist audiences.
            Client Focus: Dynamically tailor reports in response to client feedback, ensuring the final analysis aligns with their strategic objectives.
            Adherence to Excellence: Maintain the highest standards of quality and integrity in report generation, following established benchmarks for analytical rigor.
            Performance Indicators:
            The efficacy of the Financial Analysis Report is measured by its utility in providing clear, actionable insights. This encompasses aiding corporate decision-making, pinpointing areas for operational enhancement, and offering a lucid evaluation of the company's financial health. 
            Success is ultimately reflected in the report's contribution to informed investment decisions and strategic planning.
"""

user_prompt = """
            With the tools you've been provided, write an annual report based on {company}'s {fyear} 10-k report, format it into a pdf.
            Pay attention to the followings:
            - Explicitly explain your working plan before you kick off.
            - Use tools one by one for clarity, especially when asking for instructions.  
            - Display any image in the chat once generated.
            - All the paragraphs should combine between 400 and 450 words, don't generate the pdf until this is explicitly fulfilled.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", expert_investor_system_prompt),
    ("user", user_prompt)
])

messages = prompt.format_messages(company=company, fyear=fyear)
# TODO: (Add tools)
            # FMPUtils.get_sec_report,  # Retrieve SEC report url and filing date
            # IPythonUtils.display_image,  # Display image in IPython
            # TextUtils.check_text_length,  # Check text length
            # ReportLabUtils.build_annual_report,  # Build annual report in designed pdf format
            # ReportAnalysisUtils,  # Expert Knowledge for Report Analysis
            # ReportChartUtils,  # Expert Knowledge for Report Chart Plotting
messages

[SystemMessage(content="\n            Role: Expert Investor\n            Department: Finance\n            Primary Responsibility: Generation of Customized Financial Analysis Reports\n\n            Role Description:\n            As an Expert Investor within the finance domain, your expertise is harnessed to develop bespoke Financial Analysis Reports that cater to specific client requirements. \n            This role demands a deep dive into financial statements and market data to unearth insights regarding a company's financial performance and stability. \n            Engaging directly with clients to gather essential information and continuously refining the report with their feedback ensures the final product precisely meets their needs and expectations.\n\n            Key Objectives:\n\n            Analytical Precision: Employ meticulous analytical prowess to interpret financial data, identifying underlying trends and anomalies.\n            Effective Communication: Simplify and effe

# Using ReAct agent

In [9]:
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import BaseTool, StructuredTool, tool
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage

import uuid

thread_id = str(uuid.uuid4())

config = {
    "configurable": {
        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
    }
}
llm = ChatGroq(temperature=0,model="llama3-70b-8192")

## Run the graph

In [10]:
graph = create_react_agent(llm, tools=tools)
inputs = {"messages": messages}
for s in graph.stream(inputs, config, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================


            With the tools you've been provided, write an annual report based on Microsoft's 2023 10-k report, format it into a pdf.
            Pay attention to the followings:
            - Explicitly explain your working plan before you kick off.
            - Use tools one by one for clarity, especially when asking for instructions.  
            - Display any image in the chat once generated.
            - All the paragraphs should combine between 400 and 450 words, don't generate the pdf until this is explicitly fulfilled.

================================== Ai Message ==================================
Tool Calls:
  get_sec_report (call_kxx7)
 Call ID: call_kxx7
  Args:
    ticker_symbol: MSFT
    fyear: 2023
FMP api key found successfully.
================================= Tool Message =================================
Name: get_sec_report

Link: https://www.sec.gov/Archives/edgar/data/789019/000